In [1]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

2025-12-29 23:41:57.217261: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-29 23:41:57.254177: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-29 23:41:58.207475: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:

# Load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [3]:
# Load the pre-trained model with ReLU activation
model = load_model('simple_rnn_imdb.h5')
model.summary()

I0000 00:00:1767031919.270738    7396 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1221 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
model.get_weights()

[array([[ 0.20894933,  0.84512395,  0.29243734, ..., -0.6033159 ,
         -0.73510337,  1.0943631 ],
        [-0.01917174,  0.07295189,  0.01145225, ..., -0.04728982,
         -0.04938209,  0.05952793],
        [-0.04772557,  0.21008556,  0.04166476, ..., -0.22646648,
         -0.09313596,  0.26412717],
        ...,
        [-0.03376852,  0.06616929,  0.05591362, ..., -0.04418289,
          0.07275483, -0.12633464],
        [-0.03470676, -0.00597262, -0.0483466 , ...,  0.03216488,
          0.06493462,  0.0235825 ],
        [-0.04016531,  0.15518053,  0.12873398, ...,  0.1220995 ,
          0.1025422 , -0.0964502 ]], shape=(10000, 128), dtype=float32),
 array([[-0.15459205,  0.14330857, -0.00278782, ...,  0.03284907,
         -0.13958038, -0.06072861],
        [-0.00158582, -0.07688911, -0.08009917, ..., -0.11655059,
          0.02110804, -0.00690086],
        [ 0.07598489,  0.15336268,  0.11573537, ...,  0.05035895,
         -0.02213918, -0.06003958],
        ...,
        [-0.0426321

In [5]:
# Step 2: Helper Functions
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

In [6]:
### Prediction  function

def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)

    prediction=model.predict(preprocessed_input)

    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    
    return sentiment, prediction[0][0]



In [7]:
# Step 4: User Input and Prediction
# Example review for prediction
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."

sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

2025-12-29 23:42:00.031608: I external/local_xla/xla/service/service.cc:163] XLA service 0x5c2973ed7430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-29 23:42:00.031626: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
Review: This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Positive
Prediction Score: 0.718357503414154


2025-12-29 23:42:00.041749: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-29 23:42:00.067813: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91701
I0000 00:00:1767031920.170072    7472 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
